In [8]:
from sklearn.svm import SVC
import json
import pandas as pd

In [9]:
with open("CS4300sp2021-ljw222-rsp99-mot23-rgb226-ss2698/finalData2.json", "r") as f:
    data = json.load(f)

In [10]:
# each row of DF:
# - restaurant
# - text of all reviews together?
# - columns for each ambiance label, 1 or 0

df = pd.DataFrame(columns=['restaurant', 'reviews', 'touristy', 'classy', 'romantic', 'casual', 
                          'hipster', 'divey', 'intimate', 'trendy', 'upscale'])

In [15]:
for restaurant, restaurant_dict in data['BOSTON'].items():
    ambiances = restaurant_dict['ambience']
    review_texts = [r['text'] for r in restaurant_dict['reviews']]
    review_combined = '\n'.join(review_texts)
    new_row = dict()
    new_row['restaurant'] = restaurant
    new_row['reviews'] = review_combined
    for label in ['touristy', 'classy', 'romantic', 'casual', 'hipster', 'divey', 'intimate', 'trendy', 'upscale']:
        if label in ambiances:
            new_row[label] = 1
        else:
            new_row[label] = 0
    df = df.append(new_row, ignore_index=True)

In [16]:
df.head()

,restaurant,reviews,touristy,classy,romantic,casual,hipster,divey,intimate,trendy,upscale
0,Legal Sea Foods,"""If it isn't fresh, it isn't legal, "" is Legal...",1,0,0,0,0,0,0,0,0
1,Espresso Minute,Very good coffee and solid crepes. I would adm...,0,0,0,0,0,0,0,0,0
2,Amrheins,Being seated within 5 mins upon our arrival an...,0,0,0,1,0,0,0,0,0
3,Fiouna's Persian Fusion Cuisine,This one star review...I don't like writing th...,0,0,0,1,0,0,0,0,0
4,Code 10,The reason why I'm giving this place a 2 at al...,0,0,0,1,0,0,0,0,0


In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [18]:
X = df['reviews']
Y = df.drop(['restaurant', 'reviews'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)
vec = TfidfVectorizer()
Xtrain_features = vec.fit_transform(X_train)
Xtest_features = vec.transform(X_test)

In [29]:
svm = SVC()
svm.fit(Xtrain_features, y_train['casual'].astype('int'))
preds = svm.predict(Xtest_features)

In [24]:
from sklearn.metrics import classification_report, confusion_matrix

In [30]:
confusion_matrix(y_test['casual'].astype('int'), preds)

array([[371,  29],
       [215,  53]], dtype=int64)

In [31]:
print(classification_report(y_test['casual'].astype('int'), preds))

              precision    recall  f1-score   support

           0       0.63      0.93      0.75       400
           1       0.65      0.20      0.30       268

    accuracy                           0.63       668
   macro avg       0.64      0.56      0.53       668
weighted avg       0.64      0.63      0.57       668

